# Exploratory Data Analysis of IMDB Dataset

This notebook provides a comprehensive exploratory data analysis (EDA) of a movie dataset, utilized within the context of a data science project. The analysis is designed to uncover key insights and patterns within the data, followed by a series of relevant questions to deepen understanding. The project will conclude with the development of a predictive model

The  objective of this challenge extends beyond descriptive statistics; it aims to demonstrate effective problem-solving skills and creative thinking in addressing business-related questions. To achieve this, hypotheses will be formulated and subsequently tested through data analysis and visualizations.I will generate some hypothesis, then they will be tested by data and visualizations.

Throughout the notebook, I will write:
- **Code cells** with analysis.  
- **Markdown explanations** describing each cell and the reasoning behind each analytical step. 
- **Interpretations** sumarizing key findings derived from the results after runnig the code.  

### Step 1: Imports and setting up the environment

In [ ]:
# Imports
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Helper for display
from IPython.display import display

# Load the dataset and preview basic information

The dataset is provided in CSV format.

The dataset is loaded into a pandas DataFrame and its structure inspected with `.info()` to check data types and missing values  
Also display a random sample of rows for a first impression of the data.  

In [ ]:
df = pd.read_csv('desafio_indicium_imdb.csv')

In [ ]:
df.info()
display(df.sample(5))

### Some columns were also sampled on text format for a raw visualization of each variable separatedly. 

In [ ]:
print('\nExample raw values for problem columns:')
for c in ['Series_Title', 'Released_Year', 'Certificate', 'Runtime', 'Genre', 'IMDB_Rating', 'Meta_score', 'Director', 'No_of_Votes', 'Gross', 'Overview']:
    print(f"\n--- {c} sample ---")
    print(df[c].sample(5))

### Observations from first impressions  

- We can already infer what variables are numerical, categorical or text fields.
- Columns 'Gross', 'Meta_score' and 'Certificate' have missing values.
- Some variables need to be cleaned and converted to numerical:
    - **Released_Year**: Stored as strings.  
    - **Runtime**: Stored as strings with "min", needs conversion to integers.  
    - **Gross**: Likely stored with commas and currency symbols, must be parsed into numeric values.  
    - **Certificate** and **Genre**: Contain categorical text, will need normalization (e.g., splitting genres).

Therefore a data cleaning step will be needed before proceeding to deeper analysis. Proper preprocessing will allow reliable correlations and hypothesis testing.  

# Hypotheses


These hypotheses will be tested, verified and either proven or disproven during the analysis.

### H1: Popularity and Gross Revenue
**Hypothesis:** Higher number of votes (popularity) is associated with higher gross revenue.  
- Correlation analysis
- Scatter plot of `No_of_Votes` vs `Gross`
- Group averages by vote quantiles  


### H2: Meta Score vs IMDB Rating
**Hypothesis:** A higher `Meta_score` does not necessarily mean a higher `IMDB_Rating` (critics vs viewers).  
- Correlation analysis
- Scatter plot of `Meta_score` vs `IMDB_Rating`
- Examine and interpret outliers  


### H3: Runtime and IMDB Rating
**Hypothesis:** Runtime influences ratings — very short or very long runtimes may correlate with lower or higher IMDB ratings.  
- Bin runtimes into categories (short, medium, long)
- Compare mean IMDB ratings across bins  


### H4: Genre, Gross, and Rating Relationship
**Hypothesis:** There is a relationship between genre and both gross revenue and ratings.  
- Handle both single and multiple genres (e.g., split genre strings)
- Analyze genre combinations and their average gross and ratings
- Boxplots of log-transformed gross revenue by genre
- Compare means using t-tests or non-parametric tests  


### H5: Director and Gross
**Hypothesis:** Directors with multiple successful movies (high average gross/rating) may have predictive value.  
- Group data by director
- Compute mean gross and rating per director
- Consider filtering out directors with few movies (low sample size)
- Analyze director-wise average gross and rating
- Explore trends across multiple movies per director  


### H6: Star and Gross
**Hypothesis:** Actors (stars) influence gross and rating; combinations of directors and stars may have a compounding effect.  
- Analyze stars and directors separately
- Test combinations of stars + directors and their joint impact on gross/rating  


### H7: External Data - Streaming Trends
**Hypothesis:** Streaming platform releases (e.g., Netflix, Amazon) influence trends in gross and ratings.    
- Compare movies released on streaming platforms vs theatrical
- Examine counts, gross, and ratings by platform

---

# Data cleaning and creation of new columns
Now that the raw data was inspected, the necessary transformations can be applied and new columns can be created to prepare the dataset for analysis.

#### Cleaning of numerical columns

In [ ]:
# Cell: apply cleaning
# Gross
def parse_money(x):
    if pd.isna(x):
        return np.nan
    try:
        return float(str(x).replace(',', ''))
    except:
        return np.nan
df['Gross_clean'] = df['Gross'].apply(parse_money)

# Runtime
# no need for null checks on runtime, because as inspected, there are no missing values on this column
df['Runtime_clean'] = df["Runtime"].str.replace(" min", "").astype('Int64') 

# Year
df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce').astype('Int64')

#### Text columns
Since 'Genre' has sometimes 1, 2 or 3 listed on the same column, I created a 'Genre_list' and 'Num_genres' variables that will be useful for analysis. 

In [ ]:
# Genres
df['Genre_list'] = df['Genre'].fillna('').apply(lambda s: [g.strip() for g in s.split(',')] if s!='' else [])
df['Num_genres'] = df['Genre_list'].apply(len)

#### Quick Validation

After the data cleaning steps, the newly created columns are displayed in a sample table for inspection, and their data types are verified to ensure correctness.

In [ ]:
# Table visualization
display(df[['Series_Title', 'Runtime_clean', 'Gross_clean', 'Genre_list']].sample(5))

# Type check
print(df.dtypes)

##### Numeric columns assigned to a variable

#### Statistical summary of the new columns
To understand the general characteristics of the dataset, we compute descriptive statistics for key numeric columns:  
We use `.describe()` to obtain counts, means, standard deviations, quartiles, and ranges. This gives us a first quantitative overview and helps identify potential outliers.  

In [ ]:
numeric_cols = ['Released_Year','Runtime_clean','IMDB_Rating','Meta_score','Gross_clean','Num_genres']
df[numeric_cols].describe().transpose()

#### Observations from Data Cleaning

From the descriptive statistics:  

- **Released_Year**  
   - Most movies are recent (25th percentile = 1976, 75th percentile = 2009).  
   - The range goes back to 1920, showing some older films are included.  

- **Runtime_clean**  
   - Average runtime is ~123 minutes.  
   - Some extreme values (min = 45, max = 321) might represent unusually short or very long movies.  

- **IMDB_Rating**  
   - Ratings are tightly clustered (mean ~7.95, std ~0.27), indicating most movies are generally well-rated.  
   - Range from 7.6 to 9.2 — no very low-rated films in this dataset.  

- **Meta_score**  
   - Some missing values (count = 842 out of 999).  
   - Mean ~78, indicating mostly positive critical reception.  

- **Gross_clean**  
   - Highly skewed distribution (mean = 68M, max = 936M).  
   - The large standard deviation suggests strong outliers (blockbusters).  

- **Num_genres**  
   - Most movies have 2–3 genres; single-genre movies are rare.

With the data cleaned and initial checks complete, we can now move on to **visual exploration of numeric variables**, which will help reveal patterns, distributions, and potential relationships between movie characteristics and box office success.  

# Plotting of numeric variables
#### Distribution of IMDB_Rating, Meta_score, Runtime and Gross
With the dataset cleaned and modified, we now turn to visual exploration of numeric variables.  
This helps us understand the **distribution and variability** of key movie features.  

We’ll focus on:  
- **IMDB Rating** (audience evaluation)  
- **Meta Score** (critic evaluation)  
- **Runtime** (movie duration in minutes)  
- **Gross** (box office revenue, log-transformed to reduce skewness)  

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 10))  # Create 2x2 grid

# Distribution of IMDB ratings
sns.histplot(df["IMDB_Rating"], bins=20, kde=True, ax=axs[0, 0])
axs[0, 0].set_title("Distribution of IMDB Ratings")

# Distribution of Meta score
sns.histplot(df['Meta_score'].dropna(), bins=20, kde=True, ax=axs[0, 1])
axs[0, 1].set_title('Meta score distribution')
axs[0, 1].set_xlabel('Meta_score')

# Distribution of Runtime
sns.histplot(df["Runtime_clean"], bins=50, kde=True, ax=axs[1, 0])
axs[1, 0].set_title("Distribution of Movie Runtimes")

# Distribution of Gross (logarithmic)
sns.histplot(np.log1p(df['Gross_clean'].dropna()), bins=50, kde=True)
plt.title('Distribution of log(1 + Gross)')
plt.show()

plt.tight_layout()  # Adjust spacing between plots
plt.show()

#### Releases per Year
Since there is too many years and I wanted to show them all, i made a bigger graphic with bars for each year from 1920 to 2020.

In [ ]:
# Movie releases over time
plt.figure(figsize=(14, 8))
sns.countplot(x="Released_Year", data=df, order=sorted(df["Released_Year"].dropna().unique()))
plt.xticks(rotation=90,fontsize=8)
plt.title("Number of Movies Released per Year")
plt.show()

#### Observations

- **IMDB Ratings**: The histogram is heavily concentrated between 7.7 and 8.1, and a distribution with very little variation. Outliers on the high side (9.0–9.2) correspond to classics such as The Dark Knight and Pulp Fiction. This indicates the dataset is curated toward high-quality films.

- **Meta Score**:The histogram shows a roughly normal distribution, centered around 75–80, with tails down to ~30. A cluster at the top (90–100) indicates critically acclaimed films

- **Runtime**:Runtimes vary between 110–130 minutes, with a mode expected around 120 min. Some values are quite long.

- **Gross**:The Gross log histogram smooths the extreme skew, showing a more balanced spread. A few mega-hits (500M) dominate the high end, while most films cluster around 50M–100M.

- **Released_Year**: The count plot shows a sharp increase from 1990 onward, peaking in the 2000s and 2010s. Very few films appear before 1960. This confirms a sampling bias: the dataset is not representative of all movies but of notable/recognized ones. Year must always be included as a control to avoid spurious trends.

**Important**: Gross must be inflation-adjusted for fair comparisons across decades. Without that, older movies may look artificially “small” compared to modern blockbusters.

# Exploring Categorical Columns

Now, the categorical attributes will be explored in text form. The vizualizations will be showed when doing the bivariate analysis.
Checking the most frequent values for:  
- **Directors**  
- **Actors (Star1, Star2, Star3)**  
- **Certificates**  
- **Genres**  

### Directors and Stars

In [ ]:
print('\nTop directors:')
display(df['Director'].value_counts().head(10))

print('\nTop Star1 actors:')
display(df['Star1'].value_counts().head(10))

print('\nTop Star2 actors:')
display(df['Star2'].value_counts().head(10))

print('\nTop Star3 actors:')
display(df['Star3'].value_counts().head(10))

### Genres

In [ ]:
genre_counts = df['Genre_list'].explode().value_counts()
print('\nTop Genres:')
display(genre_counts.head(10))

print('\nTop Genre Combinations:')
display(df['Genre'].value_counts().head(10))

### Certificates

In [ ]:
print('\nTop Certificates:')
display(df['Certificate'].value_counts())

#### Observations

- The dataset highlights many of the most influential filmmakers. This again shows the dataset is curated toward canonical cinema classics and auteur directors dominate, rather than random sampling of movies.  
Many top actors are tied to franchises, which may drive gross figures disproportionately.

- Most frequent: Drama (723), Comedy (233), Crime (209), Adventure (196), Action (189).
Drama alone accounts for ~72% of all films (often combined with other genres).  
The dataset is dominated by Drama, either solo or in combinations. Lighter genres like Animation, Romance, and Mystery are present but smaller.


- Most frequent: U (234), A (196), UA (175), and R (146).  
American ratings (PG, PG-13, R) appear, but Indian-style ratings (U, UA, A) are much more frequent.

**Hypothesis**: Genre mix likely affects both ratings and gross. For instance, dramas may dominate IMDB ratings, while action/adventure may dominate gross

# Bivariate Analysis

## Correlation Heatmap 
To understand how the main numerical variables relate to each other, we calculate the correlation matrix. This helps identify whether variables move together (positive correlation) or in opposite directions (negative correlation). Strong correlations may suggest redundancy or predictive relationships.

Hypotheses tested and verified:  
- Higher No_of_Votes (popularity) is associated with higher Gross. (Test: correlation + scatter plot + group averages by vote quantile.)  
- Higher Meta_score does not necessarily mean higher IMDB_Rating (critics vs viewers). (Test: correlation + scatter + examine outliers.)  
- Runtime influences ratings (very short/very long runtimes may correlate with lower/higher IMDB ratings). (Test: runtime bins vs mean rating.)
- Released_Year influences Meta_score (older movies might have higher Meta_score). (Test: correlation + scatter + examine outliers.)    

In [ ]:
corr = df[["Released_Year", "IMDB_Rating", "Meta_score", "No_of_Votes", "Gross_clean", "Runtime_clean"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Between Numeric Variables")
plt.show()

In [ ]:
corr = df[["Released_Year", "IMDB_Rating", "Meta_score", "No_of_Votes", "Gross_clean", "Runtime_clean"]].corr()
print(corr)

#### From the correlation matrix, we can infer that:

- IMDB Rating & No. of Votes: 0.48  
    Popular movies (more votes) tend to have slightly higher ratings.

- IMDB Rating & Meta_score: 0.27  
  There’s a small tendency for critics and audience ratings to align, but not strongly. Often they diverge.
  
- Gross_clean & No. of Votes: 0.59  
     Higher-grossing movies typically receive more votes — likely due to wider distribution and audience exposure.

- Released Year & Meta_score: -0.34  
    Older movies may have slightly higher MetaScores, or newer movies score lower on MetaScore. Could reflect shifting critic standards or nostalgia bias.

- Released Year & Gross_clean: 0.23  
  More recent movies tend to make slightly more money, likely due to inflation or larger market reach.

- Runtime & IMDB Rating: 0.24  
  Slight trend suggesting longer movies may receive better ratings. Which

- Gross_clean & IMDB Rating: 0.10  
  High-grossing movies don’t necessarily have higher IMDB ratings. Box office and quality (as perceived by users) are weakly related.

### Scatterplot of correlations
The most meaningful correlations are scattered for a better visualization of outliers and other notable patterns.
- Meta_score and Released_Year
- Gross_clean and Released_Year
- IMDB_Rating and Meta_score
- IMDB_Rating and Runtime_clean

In [ ]:
plt.figure(figsize=(13, 8))
# Meta_score vs Released_Year
sns.scatterplot(data=df, x="Released_Year", y="Meta_score", alpha=0.6)
plt.title("Meta_score vs Released_Year")
plt.show()

In [ ]:
plt.figure(figsize=(13, 8))
# Gross_clean vs Released_Year
sns.scatterplot(data=df, x="Released_Year", y="Gross_clean", alpha=0.6)
plt.title("Released_Year vs Gross_clean")
plt.show()

In [ ]:
# Create a 1x2 subplot
fig, axes = plt.subplots(1, 2, figsize=(13, 6))
# IMDB_Rating vs Meta_score
sns.scatterplot(data=df, x="IMDB_Rating", y="Meta_score", alpha=0.6, ax=axes[0])
axes[0].set_title("IMDB Rating vs Meta_score")

# IMDB_Rating vs Runtime_clean
sns.scatterplot(data=df, x="IMDB_Rating", y="Runtime_clean", alpha=0.6, ax=axes[1])
axes[1].set_title("IMDB Rating vs Runtime_clean")

#### Scatterplot insights

- Recent films show wider variation but slightly lower critic scores on average.

- Outliers: Some older movies with exceptional box office success (e.g., Titanic) still stand out against newer blockbusters.

- IMDB_Rating vs Meta_score

- Slightly positive relationship between IMDB_Rating and Meta_score, showing that audiences and critics often agree. However, scatter indicates differences in evaluation:  
    . Some movies loved by fans but not critics (high IMDB, low Meta).  
    . Some movies praised by critics but underappreciated by audiences (high Meta, lower IMDB).  
Outliers are interesting cases worth highlighting (cult classics or divisive films).

- Weak but noticeable positive association between IMDB_Rating and Runtime_clean. However, the scatter is wide, both short and long movies can achieve high or low ratings.

# Questions

### Movie Recommendation for an Unknown Viewer

To provide an objective and justifiable recommendation, we define criteria based on widely accepted indicators of movie quality and popularity, we have to think about a general audience: something widely appealing, well-received, and accessible.  
A good general recommendation should ideally satisfy:  
- High IMDb rating – indicates strong general audience approval.
- High number of votes – ensures that the rating is reliable and not from just a few people.
- Decent box office/gross – usually means it had broad appeal.
- Popular genre – something many people like, like Adventure, Comedy, Action, or Family.  

For broader appeal, movies can also be filtered by certificate or runtime, e.g. movies under 2 hours, to increase the likelihood that a general audience will enjoy them. So:
- Accessible runtime, not too long or too short.
- Avoid movies with very restrictive certifications for a general audience recommendation.

By applying this, we identify movies that are popular, highly rated, and statistically significant. These are safe recommendations for someone whose preferences are unknown.

In [ ]:
# Filter criteria
recommended = df[
    (df['IMDB_Rating'] >= 8.5) & # high rating
    (df['No_of_Votes'] >= 10000) & # enough votes
    (df['Gross_clean'].notna()) &
    (df['Runtime_clean'] >= 90) & (df['Runtime_clean'] <= 180) & # reasonable runtime
    (df['Released_Year'] >= 1990) & # not too old
    (~df['Certificate'].isin(['A', 'R'])) # reasonable restriction
]

# Sort by rating and votes
recommended = recommended.sort_values(by=['IMDB_Rating', 'No_of_Votes'], ascending=False)

# Show top recommendation
top_movie = recommended.iloc[0]
print(top_movie[['Series_Title', 'Released_Year', 'Genre', 'IMDB_Rating', 'No_of_Votes', 'Director', 'Star1']])

This is a nice recommendation that fills all the checks:   

- IMDb Rating: Very high (9+), universally recognized as excellent.  
- Number of Votes: Large enough to trust the score.  
- Genre: Action with broad appeal.  
- Standard runtime and certificate not so restricted makes it accessible for most audiences.  
- Superhero movies are widely appealing.  
- Famous celebrities.  

For someone I don’t know, I would recommend “The Dark Knight”.

### What are the main factors associated with a high box office revenue for a film?

So in this question the 'Gross' column will be compared to all the others.   
From the analysis made using the correlation matrix, we already know the numerical variables that are associated with 'Gross'.  

- No_of_Votes: 0.59 (strongest)  
- Released_Year: 0.23 (newer films tend to gross more)  
- Runtime: 0.14 (tiny positive)  

Considerations:  
Gross_clean missing for ~16.9% of films; Meta_score missing for ~15.7%.
This sample is biased toward well-rated movies (top IMDb titles), so results reflect “high-quality / popular” films rather than the full market.
The inflation can overstate the advantage of newer films.

#### Comparing Gross with text columns

We now assess categorical drivers of revenue. We group by multi-genre combinations and individual genres (after splitting), then by director and lead actor. To reduce noise, we filter to categories with adequate support (n > 10 for genres, n > 5 for directors/actors). Our hypotheses: tentpole genres, proven directors, and major stars will show higher average grosses.

##### Gross per genre

We test whether some genres systematically earn more. Because movies often have multiple genres, we analyze both:  
- Genre combos (exact strings) and
- Individual genres via Genre_list = Genre.str.split(", ") + explode.
To avoid noise, we filter to categories with n > 10.

##### Top multi-genre combos by avg gross (n>10):  
- Action, Adventure, Sci-Fi = \$281M (n=21)
- Animation, Adventure, Comedy = \$225M (n=20)
- Action, Adventure, Drama = \$222M (n=12)

##### Top single genres by avg gross (n>10):
- Adventure = \$166M (n=170)
- Sci-Fi = \$148M (n=61)
- Action = \$141M (n=156)
- Animation = \$128M (n=67)

As expected, Action/Adventure/Sci-Fi and family-friendly Animation combos dominate by average gross (e.g., Action-Adventure-Sci-Fi = 281M). For single genres, Adventure (166M), Sci-Fi (148M), Action (141M), and Animation (128M) lead—consistent with broad appeal and franchise potential.

In [ ]:
# Group by the 'Genre' column
genre_stats = (
    df.groupby("Genre")
      .agg(
          Count=('Gross_clean', 'count'),
          AvgGross=('Gross_clean', 'mean')
      )
)

# Filter out rare genres (less than 10 movies)
genre_stats = genre_stats[genre_stats['Count'] > 10].sort_values('AvgGross', ascending=False)

# Format AvgGross as currency
genre_stats['AvgGross'] = genre_stats['AvgGross'].map('${:,.2f}'.format)

# Display the top 10 genres by average gross
print(genre_stats.head(10))

##### Using Genre_list to see if the word appears

In [ ]:
# Explode Genre_list so each genre gets its own row
df_exploded = df.explode("Genre_list")

# Group by individual genres
genre_stats = (
    df_exploded.groupby("Genre_list")
    .agg(
        Count=('Gross_clean', 'count'),
        AvgGross=('Gross_clean', 'mean')
    )
)

genre_stats = genre_stats[genre_stats['Count'] > 10].sort_values('AvgGross', ascending=False)

genre_stats['AvgGross'] = genre_stats['AvgGross'].map('${:,.2f}'.format)

print(genre_stats.head(10))

##### Gross per Director and Star1

We check whether certain directors or lead actors (Star1) are associated with higher grosses. To minimize small-sample noise, we keep creators with >5 films in the dataset.  

##### Directors:  
- Christopher Nolan — avg = \$242M (n=8)
- Steven Spielberg — avg = \$191M (n=13)
- Ridley Scott — avg = \$128M (n=6)

Directors with consistent blockbuster access (e.g., Christopher Nolan ≈ 242M avg; Steven Spielberg ≈ 191M) are associated with higher grosses, likely capturing budget, IP, and marketing scale in addition to brand reputation. 

##### Lead actors:
- Daniel Radcliffe — avg = \$305M (n=9)
- Johnny Depp — avg = \$61M (n=12)
- Christian Bale — avg = \$169M (n=8)

Celebrity power appears associated with higher grosses, though it’s often entangled with genre and director, again, plausibly reflecting access to larger commercial projects.

#### Director

In [ ]:
# Group by 'Director'
genre_stats = (
    df.groupby("Director")
      .agg(
          Count=('Gross_clean', 'count'),
          AvgGross=('Gross_clean', 'mean')
      )
)

genre_stats = genre_stats[genre_stats['Count'] > 5].sort_values('AvgGross', ascending=False)

genre_stats['AvgGross'] = genre_stats['AvgGross'].map('${:,.2f}'.format)

print(genre_stats.head(10))

#### Star1

In [ ]:
# Group by 'Star1'
genre_stats = (
    df.groupby("Star1")
      .agg(
          Count=('Gross_clean', 'count'),
          AvgGross=('Gross_clean', 'mean')
      )
)

genre_stats = genre_stats[genre_stats['Count'] > 5].sort_values('AvgGross', ascending=False)

genre_stats['AvgGross'] = genre_stats['AvgGross'].map('${:,.2f}'.format)

print(genre_stats.head(10))

##### Gross per Certificate

Broad-audience categories are associated with higher grosses.  
UA/U (parental guidance/family-friendly) have much higher averages than R/A.

In [ ]:
# Group by 'Certificate'
genre_stats = (
    df.groupby("Certificate")
      .agg(
          Count=('Gross_clean', 'count'),
          AvgGross=('Gross_clean', 'mean')
      )
      .sort_values('AvgGross', ascending=False)
)

genre_stats['AvgGross'] = genre_stats['AvgGross'].map('${:,.2f}'.format)

print(genre_stats.head(10))

#### Conclusion

Answering the question, what drives the box office in this dataset are these factors:

- Reach/awareness (No_of_Votes) is the clearest driver. Marketing scale, distribution breadth, and franchise fandom matter most.
- Commercial genres (Action/Adventure/Sci-Fi) are associated with higher revenue—broad appeal + effects + franchiseability.
- Proven directors and top-billed stars correlate with higher grosses—likely because they attract budgets, marketing, and global releases.
- Release year / market scale helps (inflation, global box office)—but the effect is second-order after popularity.
- Ratings and runtime have weak direct links; great movies can underperform commercially and vice-versa.

### What insights can be derived from the "Overview" column? Is it possible to infer the genre of the movie from this column?

##### Exploring the "Overview" Column by Genre

The Overview column contains short text descriptions of each movie’s plot.
To understand whether these texts can help us infer genres, I applied text preprocessing and extracted the most frequent words per genre.

**Preprocessing Steps**

- Converted all text to lowercase (to avoid duplicates like "Love" vs. "love").
- Removed punctuation and numbers, keeping only alphabetic characters.
- Applied a Bag of Words (CountVectorizer) representation with the top 5000 most frequent words.
- Excluded common English stopwords (e.g., "the", "and", "is").
This allowed us to convert the free-text "Overview" into structured features.

**Genre Filtering**

Movies often belong to multiple genres (e.g., "Comedy, Drama"). So, similarly as I did by creating 'Genre_list', I split the Genre column and built counters to identify the Top 10 most frequent genres in the dataset.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# Preprocessing helper
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove punctuation/numbers
    return text

# Clean Overview
df['Overview_clean'] = df['Overview'].fillna('').apply(clean_text)

# Bag of Words
vectorizer = CountVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['Overview_clean'])
bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Count movies per genre
genre_counter = Counter()
for g in df['Genre'].dropna():
    for genre in g.split(','):
        genre_counter[genre.strip()] += 1

# Top 10 genres
top_genres = [g for g,_ in genre_counter.most_common(10)]
print("Top 10 genres:", top_genres)

# Print top 10 most frequent words for each top genre
print("\nTop 10 words per genre:\n")
for genre in top_genres:
    mask = df['Genre'].str.contains(genre, na=False)
    genre_words = bow[mask].sum().sort_values(ascending=False)
    top_words = genre_words.head(20).index.tolist()
    print(f"{genre}: {', '.join(top_words)}")
    print('---')

#### Answering the Question

1. The Overview column provides a textual summary of each movie, describing its plot, characters, setting, and main events. By analyzing the most frequent words in the overviews for the top 10 genres, we can identify common themes and elements associated with each genre.

    For example:
    - Drama: Words like young, man, life, world, story, family indicate that drama movies often focus on human relationships, personal struggles, and life events.
    - Comedy: Words like friends, funny, school, girl, family suggest humorous, lighthearted situations often centered on social interactions.
    - Crime: Words like murder, police, killer, crime, investigation point to themes of law, criminal activity, and suspense.
    - Adventure / Action: Words such as war, journey, team, save, battle show that these movies involve quests, conflict, and high-stakes scenarios.
    - Romance: Words like love, woman, man, falls, friendship highlight relationship-focused plots.
    - Animation: Words like princess, toy, home, friends, quest indicate family-friendly, imaginative stories.
    
    These frequent words summarize the core themes, characters, and plot elements in each genre and highlight the distinct “signature vocabulary” of different types of movies.

2. Can Genre Be Inferred from the Overview?

    Yes. The analysis shows that each genre has distinctive keywords that appear repeatedly in its overviews, crime movies consistently mention words related to law, murder, and investigation, romance and drama frequently contain words about love, family, and relationships, adventure and action contain words about journeys, battles, or missions.
    
    This implies that it is possible to infer the genre from the overview, at least to some extent. A text-based model (like a bag-of-words or TF-IDF classifier) could be trained on these overviews to predict genres, especially for genres with strong thematic vocabulary.
    
    By analyzing frequent words, we can identify the genre with reasonable accuracy and gain insights into the plot structure, character focus, and typical story elements for each type of movie.

# Prediction Model

### HistGradientBoostingRegressor

Since the target variable (`IMDB_Rating`) is continuous, this is a regression problem.  
For this task, I selected the **HistGradientBoostingRegressor**, an implementation of gradient boosting that is optimized for speed and efficiency. It is particularly well-suited for structured/tabular datasets with a mix of numerical and categorical features, like the IMDb dataset.

From the EDA, the variables that orrelate with IMDB_Rating are:

- **Numerical predictors:**  
    - Released_Year: could matter (older classics vs newer).
    - Runtime: must be converted from "142 min" → integer minutes.
    - Meta_score: strongly correlated with IMDB_Rating.
    - No_of_Votes: popularity tends to correlate with higher ratings.
    - Gross:  needs cleaning (remove commas, convert to float).  


- **Categorical predictors:**  
    - Certificate: encode as categorical.
    - Genre: split into multiple genres (multi-hot encoding).
    - Director, Stars: high-cardinality; use frequency encoding or just consider top N frequent ones.


**Variables Transformations and Preprocessing:**
- Cleaned and transformed `Runtime`, `Gross`, and `Released_Year` into numeric values.
- Scaled numerical variables (`Released_Year`, `Runtime`, `Meta_score`, `No_of_Votes`, `Gross`).
- One-hot encoded categorical variables (`Certificate`, `Genre`, `Director`, `Star1–Star4`).
- Built a preprocessing pipeline with `ColumnTransformer` to ensure reproducibility and avoid data leakage.


**Choice of model**
This model uses histogram-based binning, making training significantly faster and more memory-efficient than traditional gradient boosting, also the model natively supports missing values in numerical features, which makes preprocessing simpler. It handles non-linear relationships and complex feature interactions better than linear models.

**Model Parameters:**
- `max_iter=300`: number of boosting iterations.
- `learning_rate=0.05`: shrinkage factor, balancing bias and variance.
- `max_depth=6`: controls tree complexity to prevent overfitting.
- `random_state=42`: ensures reproducibility.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.metrics import r2_score
import joblib

# Load data
df = pd.read_csv("desafio_indicium_imdb.csv")

# Transformations
# Clean runtime
df['Runtime'] = df['Runtime'].str.extract(r'(\d+)').astype(float)

# Clean Gross
df['Gross'] = df['Gross'].replace(',', '', regex=True)
df['Gross'] = pd.to_numeric(df['Gross'], errors='coerce')

# Handle Released_Year
df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce')

# Select features
features = ['Released_Year','Runtime','Certificate','Genre','Meta_score',
            'Director','Star1','Star2','Star3','Star4','No_of_Votes','Gross']
target = 'IMDB_Rating'

X = df[features]
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing
numeric_features = ['Released_Year','Runtime','Meta_score','No_of_Votes','Gross']
categorical_features = ['Certificate','Genre','Director','Star1','Star2','Star3','Star4']

# HistGradientBoosting handles missing values natively, but not raw strings.
# We'll encode categoricals as one-hot.
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Model pipeline
modelH = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', HistGradientBoostingRegressor(
        max_iter=300,        # number of boosting iterations
        learning_rate=0.05,  # shrinkage
        max_depth=6,         # tree depth
        random_state=42))
])

# Train
modelH.fit(X_train, y_train)

# Evaluate
y_pred = modelH.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"R²: {r2:.3f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.3f}")

**Evaluation Metrics:**
- **R² = 0.388** → The model explains ~39% of the variance in IMDb ratings.
- **MSE = 0.0401** → On average, squared prediction errors are ~0.04 points.
- **RMSE = 0.200** → Predictions deviate from the true IMDb rating by ~0.2 on the 1–10 scale.

Although the R² value indicates there is room for improvement, the RMSE of **0.20** shows that the model is fairly precise in predicting IMDb ratings, given the complexity of human perception of movies. This performance is acceptable as a first predictive approach, especially considering the high variability in audience preferences and qualitative factors not fully captured in the dataset.

In [ ]:
# Saving model
joblib.dump(modelH, "model_hgb.pkl")

#### Predicting IMDB Rating using HistGradientBoostingRegressor

In [ ]:
shawshank = {
    'Released_Year': 1994,
    'Runtime': 142,
    'Certificate': 'A',
    'Genre': 'Drama',
    'Meta_score': 80.0,
    'Director': 'Frank Darabont',
    'Star1': 'Tim Robbins',
    'Star2': 'Morgan Freeman',
    'Star3': 'Bob Gunton',
    'Star4': 'William Sadler',
    'No_of_Votes': 2343110,
    'Gross': 28341469
}

shawshank_df = pd.DataFrame([shawshank])
predicted_rating = modelH.predict(shawshank_df)[0]
print(f"Predicted IMDb Rating (HistGradientBR): {predicted_rating:.2f}")

### GradientBoostingRegressor

**Choice of model**
- Captures non-linearities better than linear models.
- Handles feature interactions automatically.
- Parameters used:
  - `n_estimators=500`: number of boosting stages (trees).
  - `learning_rate=0.05`: step size shrinkage to prevent overfitting.
  - `max_depth=4`: controls the complexity of each tree.
- This model requires preprocessed input (scaling + one-hot encoding), which I handled with a `ColumnTransformer` inside a pipeline.
- I had to include `X = X.dropna(); y = y.loc[X.index]`, since it does not handle well missing values.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Load data
df = pd.read_csv("desafio_indicium_imdb.csv")

# Transformations
# Clean runtime
df['Runtime'] = df['Runtime'].str.extract(r'(\d+)').astype(float)

# Clean Gross
df['Gross'] = df['Gross'].replace(',', '', regex=True)
df['Gross'] = pd.to_numeric(df['Gross'], errors='coerce')

# Handle Released_Year
df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce')

# Select features
features = ['Released_Year','Runtime','Certificate','Genre','Meta_score',
            'Director','Star1','Star2','Star3','Star4','No_of_Votes','Gross']
target = 'IMDB_Rating'

X = df[features]
y = df[target]

X = X.dropna()
y = y.loc[X.index]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing
numeric_features = ['Released_Year','Runtime','Meta_score','No_of_Votes','Gross']
categorical_features = ['Certificate','Genre','Director','Star1','Star2','Star3','Star4']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Model pipeline with Gradient Boosting
modelG = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(
        n_estimators=500,      # number of trees
        learning_rate=0.05,    # shrinkage
        max_depth=4,           # depth of each tree
        random_state=42))
])

# Train
modelG.fit(X_train, y_train)

# Evaluate
y_pred = modelG.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"R²: {r2:.3f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.3f}")

**Evaluation Metrics:**
- **R² = 0.534**: The model explains ~53% of the variance in IMDb ratings.
- **MSE = 0.0390**: On average, squared prediction errors are ~0.039 points.
- **RMSE = 0.197**: Predictions deviate from the true IMDb rating by ~0.20 on the 1–10 scale.

With an R² of 0.53, the model captures over half of the variability in IMDb ratings, indicating a moderately strong fit. The RMSE of 0.197 suggests that predictions are fairly close to actual ratings.

In [ ]:
# Saving model
joblib.dump(modelG, "model_gb.pkl")

#### Predicting IMDB Rating using GradientBoostingRegressor

In [ ]:
shawshank = {
    'Released_Year': 1994,
    'Runtime': 142,
    'Certificate': 'A',
    'Genre': 'Drama',
    'Meta_score': 80.0,
    'Director': 'Frank Darabont',
    'Star1': 'Tim Robbins',
    'Star2': 'Morgan Freeman',
    'Star3': 'Bob Gunton',
    'Star4': 'William Sadler',
    'No_of_Votes': 2343110,
    'Gross': 28341469
}

shawshank_df = pd.DataFrame([shawshank])
predicted_rating = modelG.predict(shawshank_df)[0]
print(f"Predicted IMDb Rating (GradientBR): {predicted_rating:.2f}")

**Comparison**
- While **HistGradientBoostingRegressor** is more scalable and efficient, in this dataset the classic **GradientBoostingRegressor** achieved better predictive performance, with a notably higher R² (0.534 vs. 0.388) and slightly lower RMSE (0.197 vs. 0.200).
- This difference may be explained by the relatively small dataset size, where the efficiency advantages of histogram-based methods are less relevant, and the more traditional boosting approach benefits from deeper optimization.
- Both models show reasonable predictive power, but **GradientBoostingRegressor currently provides the best fit** for this problem.

**Conclusion**  
By testing multiple boosting approaches, I was able to confirm that ensemble methods outperform simpler baselines for predicting IMDb scores. GradientBoostingRegressor will be kept as the primary model, while HistGradientBoostingRegressor remains a strong alternative for larger datasets or scenarios requiring faster training.